# 📚4.6相爱篇-ResNet18搭建实战

🔲能今天做好的事就不要等到明天。以梦为马，学习趁年华。

ResNet（Residual Network，残差网络）较好地解决了网络的加深带来的以下问题：难以训练、梯度消失、梯度相关性差的问题。在分类、检测、分割等任务大规模使用了ResNet作为网络骨架。ResNet的思想在于引入了一个深度残差框架来解决梯度消失问题， 即让卷积网络去学习残差映射，而不是期望每一个堆叠层的网络都完整 地拟合潜在的映射（拟合函数）。

## 一、本节目标
        本节将实现 18 层的深度残差网络 ResNet18。

## 二、 卷积神经网络结构
标准的 ResNet18 接受输入为 224x224 大小的图片数据，此处将 ResNet18 进行适量修整，使得它输入大小为 32x32，输出维度为 10。调整后的 ResNet18 网络结构如图 所 示。

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598808279526651615988081526.png"/>

### 2.1 残差模块实现
 通过对ResNet18 网络结构进行分析，它是由多个残差模块堆叠而成，每个残差模块含有两个卷积和1个，Skip Connection  卷积组成，因此先实现残差模块。

In [1]:
from tensorflow.keras import Sequential,layers
import tensorflow as tf
from tensorflow import keras

In [2]:
#构建残差模块
class BasicBlock(layers.Layer):
    
    def __init__(self,filter_num, stride=1):
        super( BasicBlock,self).__init__()
        #第一个卷积
        self.conv1=layers.Conv2D(filter_num,(3,3),strides=stride,padding='same')
        self.bn1 = layers.BatchNormalization()         
        self.relu = layers.Activation('relu')
        
        #第二个卷积
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')         
        self.bn2 = layers.BatchNormalization()
        
        if stride != 1:
            self.downsample = Sequential() 
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:
            self.downsample=lambda  x:x
        def call(self, inputs, training=None): 
            #d第一个卷积
            out = self.conv1(inputs)         
            out = self.bn1(out)         
            out = self.relu(out) 
            #第二个卷积
            out = self.conv2(out)         
            out = self.bn2(out) 
            #identity模块
            identity = self.downsample(inputs) 
            #两个输出直接相加
            output = layers.add([out, identity]) 
            output = tf.nn.relu(output) # 激活函数 
            return  output

### 2.2 多个残差模块重建
进一步对ResNet18 网络结构进行分析，发现其实由4个残差模块组组成，每个残差模块组含有两个残差模块。因此可通过build_resblock 一次完成多个残差模块的新建。

In [3]:
def build_resblock(self,filter_num, blocks, stride=1): 
    res_blocks = Sequential() 
    #只有第一个BasicBlock 的步长可能不为 1，实现下采样 
    res_blocks.add(BasicBlock(filter_num, stride)) 
 
    for _ in range(1, blocks):#其他 BasicBlock 步长都为 1             
        res_blocks.add(BasicBlock(filter_num, stride=1)) 
 
        return res_blocks

### 2.3 ResNet 网络搭建
通过2.1、2.2完成了基础模块的新建， 以下来实现 ResNet 网络模型。

In [4]:
#ResNet实现类
class ResNet(keras.Model):    
    def __init__(self, layer_dims, num_classes=10): # [2, 2, 2, 2]         
        super(ResNet, self).__init__() 
        # 根网络，预处理         
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),                                 
                                layers.BatchNormalization(),                                 
                                layers.Activation('relu'),                                 
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')                                 
                               ]) 
        # 堆叠 4 个 Block，每个 block 包含了多个 BasicBlock,设置步长不一样         
        self.layer1 = self.build_resblock(64,  layer_dims[0])         
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)         
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)         
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2) 
 
        # 通过 Pooling 层将高宽降低为 1x1         
        self.avgpool = layers.GlobalAveragePooling2D() 
        # 最后连接一个全连接层分类         
        self.fc = layers.Dense(num_classes) 
    
    def build_resblock(self,filter_num, blocks, stride=1): 
        res_blocks = Sequential() 
        #只有第一个BasicBlock 的步长可能不为 1，实现下采样 
        res_blocks.add(BasicBlock(filter_num, stride)) 
 
        for _ in range(1, blocks):#其他 BasicBlock 步长都为 1             
            res_blocks.add(BasicBlock(filter_num, stride=1)) 
 
        return res_blocks
 
    def call(self, inputs, training=None): 
        # 通过根网络         
        x = self.stem(inputs)
        # 一次通过 4 个模块        
        x = self.layer1(x)         
        x = self.layer2(x)         
        x = self.layer3(x)         
        x = self.layer4(x) 
 
        # 通过池化层         
        x = self.avgpool(x) 
        # 通过全连接层         
        x = self.fc(x) 
 
        return x

In [5]:
#搭建ResNet18
def resnet18(): 
    # 通过调整模块内部 BasicBlock 的数量和配置实现不同的 ResNet     
    return ResNet([2, 2, 2, 2]) 

In [7]:
model=resnet18()
inputs = tf.random.normal([4,32,32,3])
model(inputs)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[-0.08152785, -0.22968198, -0.4691579 , -0.5770834 , -0.22657984,
         0.38273072, -0.68749094,  0.7772414 ,  0.54096484, -0.08306218],
       [-0.10862568, -0.24518885, -0.46806252, -0.58655375, -0.18690278,
         0.38745815, -0.65693134,  0.7466553 ,  0.52465177, -0.07895319],
       [-0.09755629, -0.20533468, -0.45611775, -0.5825936 , -0.22040975,
         0.41761005, -0.66764075,  0.77483743,  0.54258674, -0.06990721],
       [-0.06418615, -0.20605296, -0.45402545, -0.5708656 , -0.22122994,
         0.39123595, -0.7051056 ,  0.7714207 ,  0.5349415 , -0.0797197 ]],
      dtype=float32)>